In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import glob
import os
import scipy.stats
from matplotlib import cm

In [ ]:
def get_pressure_weighted(x):
    dPref = (x.plev.values[0]-x.plev.values[-1])  #(p-ps)
    integral = []
    for i in range(len(x.plev)): #Integral of variable from P to Ps calculated as area between each pressure variable trapezoid then summed
        if i+1 < len(x.plev):
            area=((x.loc[dict(plev=x.plev.values[i])] + x.loc[dict(plev=x.plev.values[i+1])])/2)*(x.plev.values[i]-x.plev.values[i+1])
            integral.append(area)
    pw = (sum(integral))/dPref
    return(pw)

In [ ]:
def low_pass_weights(window, cutoff):
    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

wgts = low_pass_weights(41, 1/10)
weight = xr.DataArray(list(wgts), dims=['window'])

In [ ]:
def jettracker(x):
    x = x.fillna(0)
    x = x.ws.rolling(time=41, center=True).construct('window').dot(weight)
    x = x.dropna(dim='time',how='all')
    limit = np.quantile(x.values,0.9)
    x = x.where((x>=limit))
    x = x/x
    x = x.fillna(0)
    x = x.resample(time='QS-DEC').mean(dim='time',skipna=True)
    return x

In [ ]:
def rainref(pr,x):
    pr = pr.sel(lat=-34,method='nearest')
    pr = pr.sel(lon=18,method='nearest')
    pr = pr.resample(time='QS-DEC').mean(dim='time',skipna=True)
    pr = pr.sel(time=list(x.time.values))
    return pr

In [ ]:
model = 'ERA5'

In [ ]:
file = [glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/day/native/ua_*"),glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/day/native/va_*")]

In [ ]:


U = xr.open_mfdataset(file)[0]
V = xr.open_mfdataset(file)[1]
U = U.rename({'latitude':'lat'})
U = U.rename({'longitude':'lon'})
U = U.rename({'level':'plev'})
V = V.rename({'latitude':'lat'})
V = V.rename({'longitude':'lon'})
V = V.rename({'level':'plev'})
levels=[850,700]
x = np.sqrt(np.square(U.ua) + np.square(V.va))
x = x.rename({'__xarray_dataarray_variable__':'ws'})
x = x.sel(plev=slice(85000.0,70000.0))
x = x.sel(lat = slice(-15,-75))
x = xr.concat([x.sel(lon = slice(0,30)),x.sel(lon = slice(320,360))],dim='lon')
x.coords['lon'] = (x.coords['lon'] + 180) % 360 - 180
x = get_pressure_weighted(x)
x = x.sel(time=slice('1980', '2020'))
ERA5 = jetenator(x)


pr =xr.open_mfdataset(glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/day/native/pr*"))
pr = pr = pr.sel(time=slice('1980', '2020'))
ERA5_pr = rainref(pr,ERA5)
    

In [ ]:
def is_winter(month):
    return (month == 4) | (month == 5) | (month == 6) | (month == 7) | (month == 8) | (month == 9)


In [ ]:
D0 = ERA5.sel(time=slice('2015', '2017'))

clim = ERA5.where(is_winter(ERA5.time.dt.month)).mean(dim='time',skipna=True)
D0 = D0.where(is_winter(D0.time.dt.month)).mean(dim='time',skipna=True)


levels = np.linspace(-0.15,0.15, 21)

ax = plt.axes(projection=ccrs.Orthographic())
ax.coastlines()
plt.rcParams['hatch.linewidth']=0.4
plt.rcParams['hatch.color']='black'
c = ax.contourf(clim.lon,clim.lat,D0-clim,cmap='bwr_r',levels = levels,transform = ccrs.PlateCarree())
plt.title('ERA5 Day Zero Winter Anomaly')
ax.set_extent([-50, 32, -15, -85], ccrs.PlateCarree())
plt.colorbar(c)
ax.gridlines(linewidth=0.5, color='gray', alpha=0.5)
plt.savefig('../JET_OUT/Day_Zero_2D/D0_anom.pdf')
plt.savefig('../JET_OUT/Day_Zero_2D/D0_anom.png',dpi=1200)
plt.savefig('../JET_OUT/Day_Zero_2D/D0_anom.svg', format='svg', dpi=1200)
plt.show()
